#  LU3MA201 : Projet / Travail d’étude et de recherche

<!-- dom:AUTHOR: Aya Bouzidi at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Aya Bouzidi** ( L3 de Mathématiques à [Sorbonne Université](http://www.sorbonne-universite.fr/) ).

Licence <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>

# 3 Distance tangente comme algorithme de classification de chiffres manuscrits

<div id="ch:method_3"></div>

Bien que la méthode de la **SVD** nous donne une bonne précision de classification, elle classifie mal des chiffres bien écrits mais avec un degré de déviation. Voici un exemple : 


<figure>
    <center><img src="chiffre 6 mal reconnu.png" style="width:25%;max-width:800px;"></center>
    <center><figcaption>Chiffre mal reconnu par la SVD</figcaption></center>
</figure>


Notre troisième algorithme basé sur la **méthode tangente** nous permet de bien classifier ce type de chiffres en prenant en compte la déviation du vecteur image. 

Soit $v \in \mathbb{R}^{784}$ et $s(v,\alpha)$ une transformation de $v$ dans le plan de paramètre $\alpha$. ( $s(v,0)=v$ )

**Définition de distance tangente**

La distance tangente est définie comme la distance minimale entre deux courbes. Dans notre problème, il s'agit de la distance minimale entre deux images $p$ et $e$ qu'on peut approcher par $min_{\alpha_p,\alpha_e}||s(p,\alpha_p)-s(e,\alpha_e)||_2^2$.

Par le **théorème de Taylor**, on a:

$s(v,\alpha) \approx s(v,0)+\frac{ds}{d\alpha}(v,0)\alpha$ 

Donc $s(v,\alpha) \approx v + t_v\alpha$ avec $t_v=\frac{ds}{d\alpha}(v,0)$

Si $\alpha=(\alpha_{i})_i$ alors $s(v,\alpha) \approx v + \Sigma _i \frac{\partial s}{\partial \alpha_{i}}(v,0)\alpha_{i} \approx v + T_v\alpha_v^t$ avec $T_v=(\frac{\partial s}{\partial \alpha_{i}}(v,0))_i$ appelée **matrice tangente**

D'où $min_{\alpha_p,\alpha_e}||s(p,\alpha_p)-s(e,\alpha_e)||_2^2=min_{\alpha_p,\alpha_e}||(p-e)+(T_p\alpha_p^t-T_e\alpha_e^t)||_2^2=min_{\alpha_p,\alpha_e}||(p-e)-(-T_p \; T_e)(\alpha_p^t \; \alpha_e^t)^t)||_2^2$

On pose $A=(-T_p \; T_e)$ , $b=p-e$ et $\lambda=(\alpha_p^t \; \alpha_e^t)^t$

Alors $min_{\alpha_p,\alpha_e}||s(p,\alpha_p)-s(e,\alpha_e)||_2^2=min_{\lambda}||b-A\lambda||_2^2$

Soit $A=Q(R \; 0)^t=(Q_1 \; Q_2)(R \; 0)^t=Q_1R$ la **décomposition QR** de A

Alors $min_{\lambda}||b-A\lambda||_2^2=min_{\lambda}||(Q_1^tb-R\lambda \; \; \; \; Q_2^tb)^t||_2^2=min_{\lambda}[||Q_1^tb-R\lambda||_2^2+||Q_2^tb||_2^2]=||Q_2^tb||_2^2$

On veut écrire chaque image comme une fonction à deux variables: 

Chaque image $p$ peut s'écrire comme une matrice $P$ de taille 28x28. 

On cherche alors $p$ différentiable telle que $p(i,j)=P_{i,j}$ pour tout $(i,j) \in [1,28]$ .($P_{i,j}$ correspond alors à la valeur d'un pixel)

Cela est vérifié pour la fonction discontinue suivante: $p_*(x,y)=\Sigma_{i,j}P_{i,j}\delta(x-i)\delta(y-j)$

Pour avoir la continuité, on convolutionne $p_*$ avec la fonction gaussienne et on obtient $p(x,y)=\Sigma_{i,j}P_{i,j}g_{\sigma}(x-i,y-j)$. (Nous travaillons avec $\sigma=0.9$ pour notre algorithme)

<div id="ch:method_1"></div>

**Exemples de transformations importantes et leurs dérivées**

* **Translation d'axe (Ox)**: $T_X=\frac{\partial p}{\partial x}=p_x$

* **Translation d'axe (Oy)**: $T_Y=\frac{\partial p}{\partial y}=p_y$

* **Rotation de centre l'origine**: $T_X=y\frac{\partial p}{\partial x}-x\frac{\partial p}{\partial y}=yp_x-xp_y$

* **Scaling**: $T_X=x\frac{\partial p}{\partial x}+y\frac{\partial p}{\partial y}=xp_x+yp_y$

* **Transformation parallèle hyperbolique**: $T_X=x\frac{\partial p}{\partial x}-y\frac{\partial p}{\partial y}=xp_x-yp_y$

* **Transformation diagonale hyperbolique**: $T_X=y\frac{\partial p}{\partial x}+x\frac{\partial p}{\partial y}=yp_x+xp_y$

* **Thickening**: $T_X=(\frac{\partial p}{\partial x})^2+(\frac{\partial p}{\partial y})^2=(p_x)^2+(p_y)^2$

<div id="ch:method_1"></div>

Pour différentes transformations importantes du plan, notre algorithme est le suivant: 
    
* **Etape 1:** Pour chaque image de la base d'apprentissage et de la base de tests, calculer sa matrice tangente en fonction de la transformation choisie.


* **Etape 2:** Pour chaque image de la base de tests, calculer la distance tangente par rapport à toutes les images de la base d'apprentissage et la classifier selon le chiffre correspondant à l'image qui donne la plus petite distance tangente.

<div id="ch:method_1"></div>

**Les instructions suivantes permettent de charger les données de chiffres manuscrits disponibles dans les fichiers base_apprentissage.mat et base_test.mat :**

In [1]:
import scipy.io as spi
import numpy as np
import matplotlib.pyplot as plt

mat=spi.loadmat("base_apprentissage.mat")
data_train=np.transpose(mat['data'])
label_train=np.array(mat['label'])[0] #label: chiffre numérisé
label_train=label_train.astype(int) #Les labels sont stockés en flottants, on les convertit en entiers

mat = spi.loadmat("base_test.mat")
data_test = np.transpose(mat['data'])
label_test = np.array(mat['label'])[0]
label_test =label_test.astype(int)

In [2]:
tuples=[]
for i in range(28):
    for j in range(28):
        tuples+=[(i,j)]
        
tuples_x=np.array([tuples[i][0] for i in range(784)])
tuples_y=np.array([tuples[i][1] for i in range(784)])

<div id="ch:method_1"></div>

**On lisse les images:**

In [3]:
def smooth(v):
    P = np.reshape(v, (28,28))
    x, y = tuples_x, tuples_y
    return sum([P[i,j]*np.e**(-((x-i)**2+(y-j)**2)/(2*0.9**2)) for i,j in tuples])

In [4]:
smooth_train=[smooth(data_train[i]) for i in range(8000)]
smooth_test=[smooth(data_test[i]) for i in range(2000)]

<div id="ch:method_1"></div>

**On calcule les $p_x$ pour les chiffres:**

In [7]:
def T_X(v):
    P=np.reshape(v,(28,28))
    P_=np.gradient(P)[0]
    return np.reshape(P_,(1,-1))[0]

In [8]:
p_x_train=[T_X(smooth_train[i]) for i in range(8000)]
p_x_test=[T_X(smooth_test[i]) for i in range(2000)]

<div id="ch:method_1"></div>

**On calcule les p_y pour les chiffres:**

In [14]:
def T_Y(v):
    P=np.reshape(v,(28,28))
    P_=np.gradient(P)[1]
    return np.reshape(P_,(1,-1))[0]

In [15]:
p_y_train=[T_Y(smooth_train[i]) for i in range(8000)]
p_y_test=[T_Y(smooth_test[i]) for i in range(2000)]

<div id="ch:method_1"></div>

**On calcule les $p_r$, $p_s$, $p_{TPH}$, $p_{TDH}$ et $p_T$ pour les chiffres:**

In [20]:
p_r_train=tuples_y*np.array(p_x_train)-tuples_x*np.array(p_y_train)
p_r_test=tuples_y*np.array(p_x_test)-tuples_x*np.array(p_y_test)

In [21]:
p_s_train=tuples_x*np.array(p_x_train)+tuples_y*np.array(p_y_train)
p_s_test=tuples_x*np.array(p_x_test)+tuples_y*np.array(p_y_test)

In [22]:
p_TPH_train=tuples_x*np.array(p_x_train)-tuples_y*np.array(p_y_train)
p_TPH_test=tuples_x*np.array(p_x_test)-tuples_y*np.array(p_y_test)

In [23]:
p_TDH_train=tuples_y*np.array(p_x_train)+tuples_x*np.array(p_y_train)
p_TDH_test=tuples_y*np.array(p_x_test)+tuples_x*np.array(p_y_test)

In [24]:
p_T_train=np.array(p_x_train)**2+np.array(p_y_train)**2
p_T_test=np.array(p_x_test)**2+np.array(p_y_test)**2

<div id="ch:method_1"></div>

**On calcule les matrices tangentes pour les chiffres:**

In [28]:
matrix_T_train=[np.hstack([p_x_train[i].reshape(-1,1)]+[p_y_train[i].reshape(-1,1)]+[p_r_train[i].reshape(-1,1)]+[p_s_train[i].reshape(-1,1)]+[p_TPH_train[i].reshape(-1,1)]+[p_TDH_train[i].reshape(-1,1)]+[p_T_train[i].reshape(-1,1)]) for i in range(8000)]
matrix_T_test=[np.hstack([p_x_test[i].reshape(-1,1)]+[p_y_test[i].reshape(-1,1)]+[p_r_test[i].reshape(-1,1)]+[p_s_test[i].reshape(-1,1)]+[p_TPH_test[i].reshape(-1,1)]+[p_TDH_test[i].reshape(-1,1)]+[p_T_test[i].reshape(-1,1)]) for i in range(2000)]

<div id="ch:method_1"></div>

**Fonction de classification:**

In [33]:
def estim(j): #estime l'image data_test[j]
    A=[np.hstack((-matrix_T_train[i],matrix_T_test[j])) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

<div id="ch:method_1"></div>

**Resultats de precision:**

In [35]:
def precis(j):
    digits_j=[i for i in range(2000) if label_test[i]==j]
    a=len(digits_j)
    estimations=[estim(i) for i in digits_j]
    return list(estimations).count(j)/a

In [ ]:
for j in range(10):
    print('Précision pour le chiffre', j, 'est :',precis(j)*100, '%' )